In [1]:
import pandas as pd
from mesa import Agent, Model
from mesa.datacollection import DataCollector
from mesa.time import RandomActivation
from mesa.time import StagedActivation
from mesa.space import MultiGrid
import os

In [2]:
# Voter Agent
class Voter(Agent):
    def __init__(self, unique_id, model, attributes):
        super().__init__(unique_id, model)
        self.attributes = attributes

    def evaluate_candidate(self):
        min_difference = float('inf')
        chosen_candidate = None

        for candidate in self.model.candidates:
            difference = sum(abs(self.attributes[attr] - getattr(candidate, attr))
                             for attr in self.attributes if attr in candidate.attributes)
            if difference < min_difference:
                min_difference = difference
                chosen_candidate = candidate

        return "yes" if chosen_candidate == self.model.candidates[0] else "no"

# Candidate Agent
class Candidate(Agent):
    def __init__(self, unique_id, model, attributes):
        super().__init__(unique_id, model)
        for attr, value in attributes.items():
            setattr(self, attr, value)
        self.attributes = attributes

In [3]:
# Main Model
class VotingModel(Model):
    def __init__(self, turtles_data, candidate_attributes, info_level_threshold=10):
        self.schedule = RandomActivation(self)
        self.candidates = []
        self._steps = 0  # Initialize step counter

        # Add Candidate Agents
        for i, attrs in enumerate(candidate_attributes):
            candidate = Candidate(f"Candidate_{i}", self, attrs)
            self.schedule.add(candidate)
            self.candidates.append(candidate)

        # Add Voter Agents
        for _, row in turtles_data.iterrows():
            if row["information-level"] >= info_level_threshold:
                attributes = {
                    "government-intervention": row["government-intervention"],
                    "government-control-of-economy": row["government-control-of-economy"],
                    "free-trade": row["free-trade"],
                    "protectionism": row["protectionism"],
                    "pro-choice": row["pro-choice"],
                    "tax-increase-to-offset-debt-and-deficit": row["tax-increase-to-offset-debt-and-deficit"],
                    "crime-punishment-level": row["crime-punishment-level"],
                    "military-hawkishness": row["military-hawkishness"],
                    "non-military-interventions": row["non-military-interventions"],
                    "affirmative-action": row["affirmative-action"],
                    "prayer-in-schools": row["prayer-in-schools"],
                    "market-solution-to-healthcare": row["market-solution-to-healthcare"],
                    "moralistic-law": row["moralistic-law"]
                }
                voter = Voter(row["who"], self, attributes)
                self.schedule.add(voter)

        # Data Collector
        self.datacollector = DataCollector(
            agent_reporters={"Vote": lambda agent: agent.evaluate_candidate() if isinstance(agent, Voter) else None}
        )

    def step(self):
        self._steps += 1  # Increment step counter
        self.datacollector.collect(self)
        self.schedule.step()

In [4]:
if __name__ == "__main__":
    # Load your turtles_data DataFrame
    base_dir = "C:/Users/hugov/Github/Bachelor-project/Data/CSV"

    # Define the ranges for the file names
    range_1 = range(1, 21)
    range_2 = [500, 750]

    # Dictionary to store the data frames
    turtles_data = {}

    # Loop over the ranges and read the files dynamically
    for i in range_1:
        for j in range_2:
            # Construct the file name
            file_name = f"TURTLES_{i}_{j}.csv"
            file_path = os.path.join(base_dir, file_name)
        
            # Read the CSV file and store it in the dictionary
            if os.path.exists(file_path):
                data_name = f"Turtles{i}_{j}"
                turtles_data[data_name] = pd.read_csv(file_path)
            else:
                print(f"File not found: {file_path}")

    # Combine all DataFrames into one or use a specific DataFrame
    combined_turtles_data = pd.concat(turtles_data.values(), ignore_index=True)  # Option 1
    # specific_turtles_data = turtles_data.get("Turtles1_500")                  # Option 2

    # Define attributes for the candidates
    candidate_1_attributes = {
        "government-intervention": 10,
        "government-control-of-economy": 20,
        "free-trade": 30,
        "protectionism": 40,
        "pro-choice": 50,
        "tax-increase-to-offset-debt-and-deficit": 60,
        "crime-punishment-level": 70,
        "military-hawkishness": 80,
        "non-military-interventions": 90,
        "affirmative-action": 100,
        "prayer-in-schools": 110,
        "market-solution-to-healthcare": 120,
        "moralistic-law": 130
    }

    candidate_2_attributes = {
        "government-intervention": 15,
        "government-control-of-economy": 25,
        "free-trade": 35,
        "protectionism": 45,
        "pro-choice": 55,
        "tax-increase-to-offset-debt-and-deficit": 65,
        "crime-punishment-level": 75,
        "military-hawkishness": 85,
        "non-military-interventions": 95,
        "affirmative-action": 105,
        "prayer-in-schools": 115,
        "market-solution-to-healthcare": 125,
        "moralistic-law": 135
    }
    
    # Initialize the model with the desired DataFrame
    model = VotingModel(combined_turtles_data, [candidate_1_attributes, candidate_2_attributes])  # Use combined or specific

    # Run the model for 10 steps
    for i in range(10):
        model.step()

    # Save collected data to a CSV file
    agent_data = model.datacollector.get_agent_vars_dataframe()
    agent_data.to_csv("voting_results.csv")
    print("Voting results saved to voting_results.csv")


File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_1_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_2_500.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_2_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_3_500.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_3_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_4_500.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_4_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_5_500.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_5_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_6_500.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_6_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_7_500.csv
File not found: 

c:\Users\hugov\AppData\Local\Programs\Python\Python312\Lib\site-packages\mesa\time.py:82: FutureWarning: The AgentSet is experimental. It may be changed or removed in any and all future releases, including patch releases.
We would love to hear what you think about this new feature. If you have any thoughts, share them with us here: https://github.com/projectmesa/mesa/discussions/1919
  self._agents: AgentSet = AgentSet(agents, model)
C:\Users\hugov\AppData\Local\Temp\ipykernel_17384\3351320563.py:23: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  super().__init__(unique_id, model)


AttributeError: 'VotingModel' object has no attribute '_time'